In [ ]:
%pip install "torch==2.4.0"
%pip install "transformers==4.45.1" "datasets==3.0.1" "accelerate==0.34.2" "trl==0.11.1" "peft==0.13.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.3/797.3 MB 199.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 227.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 269.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 219.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 239.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 219.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 220.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 231.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 218.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: nvidia-nvtx-cu12
    Found existing installation: nvidia-nvtx-cu12 12.4.99
    Uninstalling nvidia-nvtx-cu12-12.4.99:
      Successfully uninstalled nvidia-nvtx-cu12-12.4.99
  Attempting uninsta

In [ ]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer

In [ ]:
dataset = load_dataset("iamjoon/klue-mrc-ko-rag-dataset", split="train")

system_message = """당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
-----
{search_result}"""

print("원본 데이터의 type 분포:")
for type_name in set(dataset['type']):
    print(f"{type_name}: {dataset['type'].count(type_name)}")

test_ratio = 0.8

train_data = []
test_data = []

for type_name in set(dataset['type']):
    curr_type_data = [i for i in range(len(dataset)) if dataset[i]['type'] == type_name]

    test_size = int(len(curr_type_data) * test_ratio)

    test_data.extend(curr_type_data[:test_size])
    train_data.extend(curr_type_data[test_size:])

def format_data(sample):
    search_result = "\n-----\n".join([f"문서{idx + 1}: {result}" for idx, result in enumerate(sample["search_result"])])

    return {
        "messages": [
            {
                "role": "system",
                "content": system_message.format(search_result=search_result),
            },
            {
                "role": "user",
                "content": sample["question"],
            },
            {
                "role": "assistant",
                "content": sample["answer"]
            },
        ],
    }

train_dataset = [format_data(dataset[i]) for i in train_data]
test_dataset = [format_data(dataset[i]) for i in test_data]

print(f"\n전체 데이터 분할 결과: Train {len(train_dataset)}개, Test {len(test_dataset)}개")

print("\n학습 데이터의 type 분포:")
for type_name in set(dataset['type']):
    count = sum(1 for i in train_data if dataset[i]['type'] == type_name)
    print(f"{type_name}: {count}")

print("\n테스트 데이터의 type 분포:")
for type_name in set(dataset['type']):
    count = sum(1 for i in test_data if dataset[i]['type'] == type_name)
    print(f"{type_name}: {count}")

README.md:   0%|          | 0.00/909 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/13.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1884 [00:00<?, ? examples/s]

원본 데이터의 type 분포:
paraphrased_question: 196
mrc_question: 491
no_answer: 404
mrc_question_with_1_to_4_negative: 296
synthetic_question: 497

전체 데이터 분할 결과: Train 380개, Test 1504개

학습 데이터의 type 분포:
paraphrased_question: 40
mrc_question: 99
no_answer: 81
mrc_question_with_1_to_4_negative: 60
synthetic_question: 100

테스트 데이터의 type 분포:
paraphrased_question: 156
mrc_question: 392
no_answer: 323
mrc_question_with_1_to_4_negative: 236
synthetic_question: 397


In [ ]:
print(type(train_dataset))
print(type(test_dataset))
train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)
print(type(train_dataset))
print(type(test_dataset))

<class 'list'>
<class 'list'>
<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


In [ ]:
model_id = "Qwen/Qwen2-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
def collate_fn(batch):
    new_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }

    for example in batch:
        clean_messages = []
        for message in example["messages"]:
            clean_message = {
                "role": message["role"],
                "content": message["content"]
            }
            clean_messages.append(clean_message)

        text = tokenizer.apply_chat_template(
            clean_messages,
            tokenize=False,
            add_generation_prompt=False
        ).strip()

        tokenized = tokenizer(
            text,
            truncation=True,
            max_length=max_seq_length,
            padding=False,
            return_tensors=None,
        )

        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]

        labels = [-100] * len(input_ids)

        im_start = "<|im_start|>"
        im_end = "<|im_end|>"
        assistant = "assistant"

        im_start_tokens = tokenizer.encode(im_start, add_special_tokens=False)
        im_end_tokens = tokenizer.encode(im_end, add_special_tokens=False)
        assistant_tokens = tokenizer.encode(assistant, add_special_tokens=False)

        i = 0
        while i < len(input_ids):
            if (i + len(im_start_tokens) <= len(input_ids) and
                input_ids[i:i+len(im_start_tokens)] == im_start_tokens):

                assistant_pos = i + len(im_start_tokens)
                if (assistant_pos + len(assistant_tokens) <= len(input_ids) and
                    input_ids[assistant_pos:assistant_pos+len(assistant_tokens)] == assistant_tokens):

                    current_pos = assistant_pos + len(assistant_tokens)

                    while current_pos < len(input_ids):
                        if (current_pos + len(im_end_tokens) <= len(input_ids) and
                            input_ids[current_pos:current_pos+len(im_end_tokens)] == im_end_tokens):

                            for j in range(len(im_end_tokens)):
                                labels[current_pos + j] = input_ids[current_pos + j]
                            break
                        labels[current_pos] = input_ids[current_pos]
                        current_pos += 1

                    i = current_pos

            i += 1

        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)

    max_length = max(len(ids) for ids in new_batch["input_ids"])

    for i in range(len(new_batch["input_ids"])):
        padding_length = max_length - len(new_batch["input_ids"][i])

        new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * padding_length)
        new_batch["attention_mask"][i].extend([0] * padding_length)
        new_batch["labels"][i].extend([-100] * padding_length)

    for k, v in new_batch.items():
        new_batch[k] = torch.tensor(v)

    return new_batch

In [ ]:
max_seq_length=8192


In [ ]:
prompt_lst = []
label_lst = []

for prompt in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(
        prompt, tokenize=False, add_generation_prompt=False
    )
    input = text.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
    label = text.split('<|im_start|>assistant')[1]
    prompt_lst.append(input)
    label_lst.append(label)

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import  AutoTokenizer, pipeline

In [ ]:
peft_model_id = "qwen2-7b-rag-ko/checkpoint-285"
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=fine_tuned_model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCaus

In [ ]:
eos_token = tokenizer("<|im_end|>",add_special_tokens=False)["input_ids"][0]

In [ ]:
def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
    return outputs[0]['generated_text'][len(prompt):].strip()

In [ ]:
for prompt, label in zip(prompt_lst[300:305], label_lst[300:305]):
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:623: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train 

    response:
도서전에서 관람객의 관심을 받을 것으로 예상되는 프로그램은 다음과 같습니다:

1. **'북 멘토 프로그램'**: 이 프로그램은 책과 관련된 일을 하고 싶은 사람들에게 각 분야 전문가들이 경험과 노하우를 전수해주는 멘토링 프로그램입니다. 시 창작, 번역, 북 디자인 등 다양한 분야에서 멘토링이 이루어지므로, 독자들이 직접적으로 책과 출판에 관한 실질적인 정보를 얻을 수 있습니다. 이 프로그램은 책과 관련된 직업을 찾는 사람들에게 큰 관심을 끌 것으로 예상됩니다 [[ref4]].

2. **'인문학 아카데미'**: 이 프로그램은 유시민 전 의원, 광고인 박웅현 씨 등 다양한 분야의 전문가들이 문화, 역사, 미학 등 다양한 주제에 대해 강의하는 곳입니다. 이는 독자들이 다양한 분야의 지식을 확장하고, 문화적 이해를 높이는 데 큰 도움이 될 것으로 예상됩니다. 특히, 다양한 분야의 전문가들이 참여하는 점이 관람객들에게 큰 관심을 끌 것으로 보입니다 [[ref4]].

3. **'저자와의 대화'**: 매년 진행되는 이 프로그램은 소설가, 시인 등 여러 분야의 작가들이 참여하여 독자들과 만날 수 있는 기회입니다. 작가들이 직접 책을 소개하고, 독자들과의 대화를 나누는 것은 독자들에게 큰 관심을 끌 것입니다. 특히, 이번 도서전 홍보대사로 나선 소설가 박범신 씨와 다른 작가들이 참여할 예정이므로, 작가들과의 만남을 기대할 수 있습니다 [[ref4]].

4. **'인도의 영혼들' 전시**: 인도는 역대 최대 규모의 주빈국 행사를 마련하여, 노벨문학상 수상 100주년을 맞는 대문호 라빈드라나트 타고르를 비롯해 여러 인도 출신 노벨상 수상자와 관련 도서를 소개하는 전시가 예정되어 있습니다. 이는 독자들이 다양한 문화와 역사에 대한 이해를 높이는 데 큰 도움이 될 것으로 예상됩니다 [[ref4]].

이와 같은 프로그램들은 각기 독자들에게 다양한 경험과 지식을 제공할 것이며, 관람객들의 관심을 끌 것으로 예상됩니다.
    label:

도서전에서 관

In [ ]:

pipe_base = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
for prompt, label in zip(prompt_lst[300:305], label_lst[300:305]):
    print(f"    response:\n{test_inference(pipe_base, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

    response:
도서전에서 관람객의 관심을 받을 것으로 예상되는 프로그램은 여러 가지가 있을 수 있습니다. 

첫째로, '저자와의 대화' 프로그램이 주목받을 수 있습니다. 이 프로그램은 소설가 박범신 씨를 비롯한 21명의 작가들이 참여하여 독자들과 만날 예정입니다. 이는 작가와 독자 간의 직접적인 소통을 제공하며, 독자들이 작가의 창작 과정과 아이디어를 더 잘 이해하는 데 도움이 될 것입니다.

둘째로, '인문학 아카데미'가 관람객의 관심을 끌 것으로 예상됩니다. 이 프로그램은 유시민 전 의원, 광고인 박웅현 씨 등이 문화 역사 미학 등 다양한 분야에 대해 강의하는 것을 통해 관람객들에게 새로운 지식과 통찰력을 제공할 것입니다.

셋째로, '북 멘토 프로그램'도 관람객의 관심을 끌 것으로 예상됩니다. 이 프로그램은 책과 관련된 일을 하고 싶은 사람들에게 각 분야 전문가들이 경험과 노하우를 전수해주는 프로그램입니다. 시 창작, 번역, 북 디자인 등의 분야에서 멘토링이 이뤄지므로, 이에 관심이 있는 관람객들에게 큰 호응을 얻을 수 있습니다.

마지막으로, 주빈국인 인도의 '인도의 영혼들' 전시와 '인도 영화 100년사' 전시도 관람객의 관심을 끌 것으로 예상됩니다. 이 전시는 인도의 문화와 역사에 대한 깊은 이해를 제공하며, 특히 노벨상 수상자들의 작품을 통해 인도 문학의 풍부함을 보여줄 것입니다. 또한 인도 영화 '데브다스'의 상영 역시 관람객들에게 인도의 영화 문화에 대한 이해를 높일 수 있을 것입니다.
    label:

도서전에서 관람객의 관심을 받을 것으로 예상되는 프로그램은 다음과 같습니다:

1. **저자와의 대화**: 소설가 박범신, 정유정, 이인화, 최민석, 김혜나, 신달자, 함민복 등 21명의 작가들이 참여하여 독자들과 직접 만나는 프로그램입니다. 이는 독자들이 좋아하는 작가와 직접 소통할 수 있는 기회를 제공하여 큰 관심을 받을 것으로 예상됩니다 [[ref3]], [[ref5]].

2. **인문학 아카데미**: 유시민 전 의원, 광

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# 경로 설정
base_model_path = "Qwen/Qwen2-7B-Instruct"
adapter_path = "./qwen2-7b-rag-ko/checkpoint-285"
merged_model_path = "./Qwen2-7B-Instruct_RAG_KO_custom_20250804"

# 디바이스 설정
device_arg = {"device_map": "auto"}


In [ ]:
# LoRA 어댑터 로드 및 병합
print(f"Loading and merging PEFT from: {adapter_path}")
peft_model = PeftModel.from_pretrained(model, adapter_path)
merged_model = peft_model.merge_and_unload()


Loading and merging PEFT from: ./qwen2-7b-rag-ko/checkpoint-285


In [ ]:
# 저장
print(f"Saving merged model to: {merged_model_path}")
model.save_pretrained(merged_model_path)
tokenizer.save_pretrained(merged_model_path)
print("✅ 모델과 토크나이저 저장 완료")


Saving merged model to: ./Qwen2-7B-Instruct_RAG_KO_custom_20250804
✅ 모델과 토크나이저 저장 완료


In [ ]:
def get_token():
  with open('20250801_Huggingfacie_Token.key', 'r', encoding='utf-8') as file:
    return file.readline().rstrip()

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

username = "mypsyche98"


In [ ]:
api.create_repo(
    token=get_token(),
    repo_id=f"{username}/Qwen2-7B-Instruct_RAG_KO_custom_20250804",
    repo_type="model"
)


RepoUrl('https://huggingface.co/mypsyche98/Qwen2-7B-Instruct_RAG_KO_custom_20250804', endpoint='https://huggingface.co', repo_type='model', repo_id='mypsyche98/Qwen2-7B-Instruct_RAG_KO_custom_20250804')

In [ ]:
api.upload_folder(
    token=get_token(),
    repo_id=f"{username}/Qwen2-7B-Instruct_RAG_KO_custom_20250804",
    folder_path=merged_model_path,
)

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...G_KO_custom_20250804/tokenizer.json:   0%|          | 24.0kB / 11.4MB            

  ...04/model-00004-of-00004.safetensors:   5%|4         | 50.3MB / 1.09GB            

  ...04/model-00003-of-00004.safetensors:   1%|1         | 58.7MB / 4.33GB            

  ...04/model-00001-of-00004.safetensors:   2%|1         | 92.3MB / 4.88GB            

  ...04/model-00002-of-00004.safetensors:   1%|1         | 58.7MB / 4.93GB            

CommitInfo(commit_url='https://huggingface.co/mypsyche98/Qwen2-7B-Instruct_RAG_KO_custom_20250804/commit/bb8c1098bd8ed496d9c6b2db898c0d77c7905d50', commit_message='Upload folder using huggingface_hub', commit_description='', oid='bb8c1098bd8ed496d9c6b2db898c0d77c7905d50', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mypsyche98/Qwen2-7B-Instruct_RAG_KO_custom_20250804', endpoint='https://huggingface.co', repo_type='model', repo_id='mypsyche98/Qwen2-7B-Instruct_RAG_KO_custom_20250804'), pr_revision=None, pr_num=None)